# MODELO BASE - SEM GENERO E SEM UTILIZADOR: DUPLA OPTIMIZAÇÃO

# POR ACABAR !!!!!!!!!!

In [23]:
import pandas as pd
import numpy as np
import json
import warnings

#warnings.filterwarnings("ignore")

# Load the data

In [24]:
df_train = pd.read_csv("data/data_train.csv")
df_test = pd.read_csv("data/data_test.csv")

In [25]:
with open("data/data_ids.json") as f:
    ids = json.load(f)

max_movieId = max(ids["moviesIDs"].values())
max_userId = max(ids["userIDs"].values())

# Create a matrix of users and movies
base_matrix = np.zeros((max_movieId + 1, max_userId + 1))
base_matrix.shape

(9633, 385)

# Define functions

In [26]:
# Utility matrix and Binary matrix
from scipy.sparse import lil_matrix

def utility_matrix_AND_binary_matrix(dfTrain, zerosMatrix = base_matrix, jsonIds = ids):
    Y = lil_matrix(zerosMatrix.shape)  # Use sparse matrix format
    R = lil_matrix(zerosMatrix.shape)
    for i, row in dfTrain.iterrows():
        original_movieId, original_userId = int(row["movieId"]), int(row["userId"])
        movieId = jsonIds["moviesIDs"][str(original_movieId)]
        userId = jsonIds["userIDs"][str(original_userId)]
        Y[movieId, userId] = row["rating"]
        R[movieId, userId] = 1
    return Y, R

In [27]:
def cofiCostFunc(params, Y, R, num_users, num_movies, num_features, Lambda):
    """
    Returns the cost and gradient for the collaborative filtering
    Lambda - regularization parameter
    """
        
    # Unfold the params
    X = params[:num_movies*num_features].reshape(num_movies,num_features)
    Theta = params[num_movies*num_features:].reshape(num_users,num_features)
    
    predictions =  X @ Theta.T
    err = (predictions - Y)
    J = 1/2 * np.sum((err**2) * R)
    
    #compute regularized cost function
    reg_X =  Lambda/2 * np.sum(Theta**2)
    reg_Theta = Lambda/2 *np.sum(X**2)
    reg_J = J + reg_X + reg_Theta
    
    # Compute gradient
    X_grad = err*R @ Theta
    Theta_grad = (err*R).T @ X
    grad = np.append(X_grad.flatten(),Theta_grad.flatten())
    
    # Compute regularized gradient
    reg_X_grad = X_grad + Lambda*X
    reg_Theta_grad = Theta_grad + Lambda*Theta
    reg_grad = np.append(reg_X_grad.flatten(),reg_Theta_grad.flatten())
    
    return J, grad, reg_J, reg_grad

In [29]:
def gradientDescent(initial_parameters,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda):
    """
    alpha - learning rate
    Optimize X and Theta
    """
    # unfold the initial parameters (consult function cofiCostFunc)
    X = initial_parameters[:num_movies*num_features].reshape(num_movies,num_features)
    Theta = initial_parameters[num_movies*num_features:].reshape(num_users,num_features)
    
    J_history =[]
    
    for i in range(num_iters):
        #Append into a single vector params X and Theta (see above)
        params = np.append(X.flatten(),Theta.flatten())        
        cost, grad = cofiCostFunc(params, Y, R, num_users, num_movies, num_features, Lambda)[2:]
        
        # unfold grad
        X_grad = grad[:num_movies*num_features].reshape(num_movies,num_features)
        Theta_grad = grad[num_movies*num_features:].reshape(num_users,num_features)
        
        #Update trainable parameters X & Theta applying gradient descent method
        X = X - alpha*X_grad
        
        Theta = Theta - alpha*Theta_grad
        
        J_history.append(cost)
    
    #Append into a single vector paramsFinal the updated X and Theta
    paramsFinal = np.append(X.flatten(),Theta.flatten())
    return paramsFinal , J_history

# Start modelling

- split the df in 8 to do cv

- define num of features to use: 20 because we have 20 diff genres of movies (02 data_vis.ipynb)

In [30]:
# Shuffle the DataFrame randomly and split it into 8 parts
df_shuffled = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
folds = np.array_split(df_shuffled, 8)

# define the number of movies, users and features
num_movies, num_users = base_matrix.shape
num_features = 20

/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


optimize the hyperparameters of the model using 8cv and minimizing RMSE

In [31]:
Lambdas = [0, 0.1, 1, 10]
alphas = [0.0001, 0.0005, 0.001, 0.002]
num_iters = 25 #500?

combo = 1
results = {}
for Lambda in Lambdas:
    for alpha in alphas:
        print(f"Combination {combo} of {len(Lambdas)*len(alphas)} with ({Lambda}, {alpha}):", end=" ") 
        combo += 1

        avg_RMSE = 0
        for exclude_index in range(8):
            merged_df = pd.concat([part for i, part in enumerate(folds) if i != exclude_index])

            # Create the utility matrix and the binary matrix
            Y, R = utility_matrix_AND_binary_matrix(merged_df)
            Y, R = Y.toarray(), R.toarray()

            # Define the initial parameters
            X = np.random.normal(loc=0, scale=0.3, size=(num_movies, num_features))
            Theta = np.random.normal(loc=0, scale=0.3, size=(num_users, num_features))
            params = np.append(X.flatten(),Theta.flatten())

            # Apply gradient descent
            param, jhist = gradientDescent(params,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda)

            # Unfold the final parameters
            X = param[:num_movies*num_features].reshape(num_movies,num_features)
            Theta = param[num_movies*num_features:].reshape(num_users,num_features)

            # Predict all ratings of num_users for num_movies
            p = X @ Theta.T

            # Compute the RMSE
            RMSE = 0
            for i, row in folds[exclude_index].iterrows():
                movieId_matrix = ids["moviesIDs"][str(int(row["movieId"]))]
                userId_matrix = ids["userIDs"][str(int(row["userId"]))]
                RMSE += abs(row["rating"] - p[int(movieId_matrix), int(userId_matrix)])**2
            RMSE = np.sqrt(RMSE/len(folds[exclude_index]))
            print(f"CV{exclude_index+1}: {RMSE:.1f}", end=", ")
            avg_RMSE += RMSE/8
        
        results[(Lambda, alpha)] = avg_RMSE
        print("")

Combination 1 of 16 with (0, 0.0001): CV1: 3.7, CV2: 3.7, CV3: 3.6, CV4: 3.6, CV5: 3.7, CV6: 3.7, CV7: 3.6, CV8: 3.7, 
Combination 2 of 16 with (0, 0.0005): CV1: 2.7, CV2: 2.8, CV3: 2.7, CV4: 2.9, CV5: 2.8, CV6: 2.7, CV7: 2.7, CV8: 2.8, 
Combination 3 of 16 with (0, 0.001): CV1: 1.8, CV2: 1.8, CV3: 1.8, CV4: 1.7, CV5: 1.7, CV6: 1.8, CV7: 1.8, CV8: 1.8, 
Combination 4 of 16 with (0, 0.002): CV1: 2.6, CV2: 2.4, CV3: 2.6, CV4: 1.8, CV5: 2.1, CV6: 3.2, CV7: 1.9, CV8: 2.3, 
Combination 5 of 16 with (0.1, 0.0001): CV1: 3.7, CV2: 3.7, CV3: 3.6, CV4: 3.6, CV5: 3.7, CV6: 3.7, CV7: 3.7, CV8: 3.6, 
Combination 6 of 16 with (0.1, 0.0005): CV1: 2.7, CV2: 2.6, CV3: 2.6, CV4: 2.9, CV5: 2.8, CV6: 2.9, CV7: 2.6, CV8: 2.9, 
Combination 7 of 16 with (0.1, 0.001): CV1: 1.8, CV2: 1.8, CV3: 1.8, CV4: 1.8, CV5: 1.7, CV6: 1.8, CV7: 1.7, CV8: 1.7, 
Combination 8 of 16 with (0.1, 0.002): CV1: 1.9, CV2: 2.6, CV3: 3.3, CV4: 3.6, CV5: 4.4, CV6: 3.9, CV7: 2.5, CV8: 2.1, 
Combination 9 of 16 with (1, 0.0001): CV1: 3

In [35]:
results

{(0, 0.0001): np.float64(3.654772824790143),
 (0, 0.0005): np.float64(2.7536373654307833),
 (0, 0.001): np.float64(1.7592764709771669),
 (0, 0.002): np.float64(2.362311761723245),
 (0.1, 0.0001): np.float64(3.6557897214861144),
 (0.1, 0.0005): np.float64(2.748252293231398),
 (0.1, 0.001): np.float64(1.758049439111155),
 (0.1, 0.002): np.float64(3.0407161187957783),
 (1, 0.0001): np.float64(3.6514850574107416),
 (1, 0.0005): np.float64(2.7901090981236996),
 (1, 0.001): np.float64(1.7646081335505999),
 (1, 0.002): np.float64(2.7597255039571635),
 (10, 0.0001): np.float64(3.6507908174201367),
 (10, 0.0005): np.float64(2.7797766740870102),
 (10, 0.001): np.float64(1.7643812548742177),
 (10, 0.002): np.float64(2.91597749477319)}

In [38]:
#aumentar numero de iteracoes, testar mais valores de lambda! e alpha?

# escolher melhor combinação de hiperparâmetros

# TOPOGRAPHIC MAP TO VISUALIZE THE RESULTS OF THE HYPERPARAMETER OPTIMIZATION

# Evaluate the optimized model

In [32]:
import matplotlib.pyplot as plt

#plt.plot(jhist)

In [33]:
df_to_evaluate = df_test

RMSE = 0
for i, row in df_to_evaluate.iterrows():
    movieId_matrix = ids["moviesIDs"][str(int(row["movieId"]))]
    userId_matrix = ids["userIDs"][str(int(row["userId"]))]
    RMSE += abs(row["rating"] - p[int(movieId_matrix), int(userId_matrix)])**2

RMSE = np.sqrt(RMSE/len(df_to_evaluate))
RMSE

np.float64(3.7141438781282274)

In [34]:
# ...